In [6]:
import os
import numpy as np
import pandas as pd
import gc

directory = './'
file_path = './merged_sorted_data.csv'
dtypes = {
    'timestamp': np.int64,
    'Asset_ID': np.int8,
#     'Count': np.int32,
#     'Open': np.float64,
#     'High': np.float64,
#     'Low': np.float64,
    'Close': np.float64,
#     'Volume': np.float64,
#     'VWAP': np.float64,
    'Target': np.float64,
}
data = pd.read_csv(file_path, dtype=dtypes, usecols=list(dtypes.keys()))
data['Time'] = pd.to_datetime(data['timestamp'], unit='s')

file_path = os.path.join(directory, 'asset_details.csv')
details = pd.read_csv(file_path)

In [7]:
price_column = 'Close'
ids = list(details.Asset_ID)
chunks = []
for id in ids:    
    asset = data[data.Asset_ID == id].copy()
    asset.sort_values(by='Time', inplace=True)
    asset.set_index(keys='Time', inplace=True)
    asset['p1'] = asset[price_column].shift(freq='-1T')
    asset['p16'] = asset[price_column].shift(freq='-16T')
    asset['r'] = np.log(asset.p16/asset.p1)
    asset.drop(['p1', 'p16'], axis=1, inplace=True)
    asset.reset_index(inplace=True)
    chunks.append(asset)

data = pd.concat(chunks)
data.sort_values(by='Time', inplace=True)

C:\Users\igor\AppData\Local\Temp\ipykernel_4484\724874275.py:8: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  asset['p1'] = asset[price_column].shift(freq='-1T')
C:\Users\igor\AppData\Local\Temp\ipykernel_4484\724874275.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  asset['p16'] = asset[price_column].shift(freq='-16T')
C:\Users\igor\AppData\Local\Temp\ipykernel_4484\724874275.py:8: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  asset['p1'] = asset[price_column].shift(freq='-1T')
C:\Users\igor\AppData\Local\Temp\ipykernel_4484\724874275.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  asset['p16'] = asset[price_column].shift(freq='-16T')
C:\Users\igor\AppData\Local\Temp\ipykernel_4484\724874275.py:8: FutureWarning: 'T' is deprecated and will be removed in a fu

In [8]:
def calculate_target(data: pd.DataFrame, details: pd.DataFrame, price_column: str):
    ids = list(details.Asset_ID)
    asset_names = list(details.Asset_Name)
    weights = np.array(list(details.Weight))

    all_timestamps = np.sort(data['timestamp'].unique())
    targets = pd.DataFrame(index=all_timestamps)

    for i, id in enumerate(ids):
        asset = data[data.Asset_ID == id].set_index(keys='timestamp')
        price = pd.Series(index=all_timestamps, data=asset[price_column])
        target_values = (price.shift(periods=-16) / price.shift(periods=-1)) - 1
        targets[f"{id}"] = target_values.fillna(0)  # Include Asset ID in column name
    
    targets['m'] = np.average(targets.fillna(0), axis=1, weights=weights)
    
    m = targets['m']

    num = targets.multiply(m.values, axis=0).rolling(3750).mean().values
    denom = m.multiply(m.values, axis=0).rolling(3750).mean().values
    beta = np.nan_to_num(num.T / denom, nan=0., posinf=0., neginf=0.)

    targets = targets - (beta * m.values).T
    targets.drop('m', axis=1, inplace=True)

    targets.insert(0, 'timestamp', targets.index)

    return targets

In [9]:
sperma = calculate_target(data, details, price_column)
sperma.to_csv('result.csv', index=False)
print(sperma)

             timestamp         2         0         1         5         7  \
1677628800  1677628800  0.003033  0.000000  0.001398  0.002620  0.000963   
1677628860  1677628860  0.003791  0.000332  0.000945  0.001744  0.001927   
1677628920  1677628920  0.003030  0.000000  0.000853  0.001744  0.000963   
1677628980  1677628980  0.001512  0.000000  0.000226  0.000870  0.000963   
1677629040  1677629040  0.001512 -0.000332  0.000581  0.000870  0.000963   
...                ...       ...       ...       ...       ...       ...   
1709250900  1709250900  0.000000  0.000000  0.000000  0.000000  0.000000   
1709250960  1709250960  0.000000  0.000000  0.000000  0.000000  0.000000   
1709251020  1709251020  0.000000  0.000000  0.000000  0.000000  0.000000   
1709251080  1709251080  0.000000  0.000000  0.000000  0.000000  0.000000   
1709251140  1709251140  0.000000  0.000000  0.000000  0.000000  0.000000   

                   6         9        11        13        12         3  \
1677628800  0